In [ ]:
# Import libraries

import pandas as pd
import sqlite3
import numpy as np

from rdkit.Chem import AllChem
from rdkit.Chem import DataStructs
from rdkit.Chem import Descriptors
from rdkit.Chem import MolFromSmiles
from rdkit import Chem
import time

import os

import pickle
import random

from src.ligand_clustering_functions import get_ligand_scaffolds, generate_decoys_from_properties

In [ ]:
def get_results(query,chembl_db):
    """Obtains data from ChEMBL db"""
    #chembl_db = '/home/gustavo/disco_2/Trabajos_Bioinfo_3/chembl_33/chembl_33.db'
    connection = sqlite3.connect(chembl_db)
    cursor = connection.cursor()
    
    cursor.execute(query)
    result = cursor.fetchall()
    cursor.close()
    connection.close()

    return result

def generate_properties_csv(molecule_dict):
    """Generate a CSV file with the physicochemical properties of the molecules in the dictionary."""
    rows = []
    for mol_id, smiles in molecule_dict.items():
        mol = Chem.MolFromSmiles(smiles)
        if mol:
            properties = {
                'compound_id': mol_id,
                #'smiles': smiles,
                'mw': Descriptors.MolWt(mol),
                'logP': Descriptors.MolLogP(mol),
                'rot_bonds': Descriptors.NumRotatableBonds(mol),
                'h_acceptors': Descriptors.NumHAcceptors(mol),
                'h_donors': Descriptors.NumHDonors(mol),
                'charge': Chem.rdmolops.GetFormalCharge(mol)
            }
            rows.append(properties)
    df = pd.DataFrame(rows)
    df.set_index('compound_id', inplace=True)
    #df.to_csv(output_file)
    return df

In [ ]:
# SQL queries to obtain data from ChEMBL

# Obtain a table of protein–ligand pairs with the associated pChEMBL (affinity) value.
query_prot_ligs = '''select distinct molecule_dictionary.chembl_id as ligand_id,
component_sequences.accession as uniprot_id, activities.pchembl_value as pchembl, activities.activity_comment as comment,
domains.source_domain_id as pfam
            from activities join assays on activities.assay_id = assays.assay_id
            join molecule_dictionary on activities.molregno = molecule_dictionary.molregno 
			join compound_structures on molecule_dictionary.molregno = compound_structures.molregno
            join target_dictionary on assays.tid = target_dictionary.tid
			join target_components on target_dictionary.tid = target_components.tid
			join component_sequences on target_components.component_id = component_sequences.component_id
            join site_components on target_components.component_id = site_components.component_id
			join domains on site_components.domain_id = domains.domain_id
            where assays.assay_type = 'B' and
            target_dictionary.target_type = "SINGLE PROTEIN"'''

# Obtain a table of ligand–SMILES pairs.
query_ligs_smiles = '''select distinct molecule_dictionary.chembl_id as ligand_id,
compound_structures.canonical_smiles as smiles
            from activities join assays on activities.assay_id = assays.assay_id
            join molecule_dictionary on activities.molregno = molecule_dictionary.molregno 
			join compound_structures on molecule_dictionary.molregno = compound_structures.molregno
            join target_dictionary on assays.tid = target_dictionary.tid
			join target_components on target_dictionary.tid = target_components.tid
			join component_sequences on target_components.component_id = component_sequences.component_id
            join site_components on target_components.component_id = site_components.component_id
			join domains on site_components.domain_id = domains.domain_id
            where assays.assay_type = 'B' and
            target_dictionary.target_type = "SINGLE PROTEIN"'''

In [ ]:
data_dir = 'data'
os.makedirs(data_dir, exist_ok=True)

## Obtaining bioactive and inactive compounds from ChEMBL

In [ ]:
# Set directory of ChEMBL db (in sqlite format)
chembl_db = './chembl_33.db'

In [ ]:
# Obtain ligand-target pairs, with pchembl value, and convert to a table with associated metadata
prot_ligs = get_results(query_prot_ligs,chembl_db)
prot_ligs = pd.DataFrame(prot_ligs,columns=['lig','prot','pchembl','comment','pfam'])

In [ ]:
# Proteins without an associated pChEMBL value but labeled as negatives were included by assigning them a pChEMBL value of 3, thereby establishing a minimum affinity threshold.
prot_ligs.loc[(prot_ligs['comment'].isin(['Not Active','inactive','No significant effect','No Activity','No binding'])) & (prot_ligs['pchembl'].isna()),'pchembl'] = 3
# Minimum pChEMBL values are limited to 3.
prot_ligs.loc[(prot_ligs['pchembl'] < 3),'pchembl'] = 3
# Maximum pChEMBL values are limited to 10.
prot_ligs.loc[(prot_ligs['pchembl'] > 10),'pchembl'] = 10

In [ ]:
# Tag positive or negative bioactivity
positive_threshold = 6.5
negative_threshold = 4.5
prot_ligs['activity'] = prot_ligs['pchembl'].apply(lambda x: 1 if x > positive_threshold else (0 if x < negative_threshold else np.nan))

# Drop undefined activity pairs
prot_ligs = prot_ligs.dropna(subset=['activity'])

In [ ]:
# Save dataset:
prot_ligs.to_csv(f'{data_dir}/prot_ligs_db.csv',index=False)

## Obtaining compound databases

In [ ]:
# Obtain compound-SMILES dictionary
ligs_smiles = get_results(query_ligs_smiles,chembl_db)
ligs_smiles = {l[0]:l[1] for l in ligs_smiles}

In [ ]:
# Save compound-SMILES dictionary
with open(f'{data_dir}/comps_smiles.pkl','wb') as f:
    pickle.dump(ligs_smiles,f)

In [ ]:
# Generate precalculated database of ligands and fingerprints
fp_size = 256
ligs_fps = {c:AllChem.GetMorganFingerprintAsBitVect(MolFromSmiles(ligs_smiles[c]),2,fp_size) for c in ligs_smiles}

In [ ]:
# Save database of compounds and fps
with open(f'{data_dir}/comps_fps.pkl','wb') as f:
    pickle.dump(ligs_fps,f)

In [ ]:
# Generate database of compounds with required properties to generate decoys
ligs_props_decoys = generate_properties_csv(ligs_smiles)

In [ ]:
# Save database of compound properties
ligs_props_decoys.to_csv(f'{data_dir}/ligs_props.csv')

In [ ]:
# Precalculate Bemis-Murcko scaffolds for all ChEMBL ligands
scaffolds = get_ligand_scaffolds(ligs_smiles)
with open(f'{data_dir}/ligs_scaffolds.pkl','wb') as f:
    pickle.dump(scaffolds,f)

## Generate decoys for each active ligand from ChEMBL

In [ ]:
# Get active ligands
actives = list(prot_ligs[prot_ligs['activity']==1]['lig'].unique())

In [ ]:
# Select Tanimoto threshold to retrieve decoys
decoy_threshold = 0.3

# Generate decoys sets for each active
i = 0
decoys_d = {}
for l in actives:
    decoys = generate_decoys_from_properties(l, ligs_props_decoys, ligs_fps, scaffolds,threshold=decoy_threshold)
    decoys_d[l] = decoys
    i += 1
    if i % 10000 == 0:
        print(f"{i} processed compounds")

In [ ]:
# Save compound and respective decoys database
with open(f'{data_dir}/decoys_dict.pkl','wb') as f:
    pickle.dump(decoys_d,f)
